## Load model

In [ ]:
import gensim
# Load Google's pre-trained Word2Vec model.
model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)
print('done')

## Unknown words mappings and clean_text function

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
from nltk.tag import pos_tag
import nltk
from nltk.corpus import stopwords
# nltk.download()

TAG_REMOVE = re.compile(r'<[^>]+>')
mapping = [('twenti ','twentieth '),('thirti ','thirtieth '),('forti ','fortieth  '),('fifti ','fiftieth '),('sixti ','sixtieth '),('seventi ','seventieth '),('eighti ','eightieth '),('nineti ','ninetieth '),
           ('arrogancy ','arrogance '),('tortur ','torture '),('desirest','desire'),('desirest ','desire '),('desir ','desire '),('receivedst ','received '),('carri ','carries '),('forgiv ','forgives '),('divid ','divides '),
           ('athirst ','thirsty '),('testifi ','testifies '),('reprove ','reprove '),('grey ','gray '),('aveng ','avenge '),('behoov ','behoove '),('reprov ','reprove '),('affrighted ','frightened '),
           ('leathern ','leather '),('abhorr ','abhor '),('rejoic ','rejoice '),('provok ','provoke '),('straiten ','straighten '),('cumbered ','encumbered '),('suffic ','suffice '),('sware ','swore '),
           ('justifi ','justify '),('decre ','decree '),('ceas  ','cease '),('mingl ','mingle '),('persuad ','persuade '),('entic ','entice '),('trembl ','tremble '),('cimeter','scimitar'),
           ('condemneth','condemn'),('prophesi ','prophesy '),('awak ','awake '),('canst ','cannot '),('engraven','engraved'),('hadst ','had '),('rebell ','rebels '),('persecut ','persecute '),
           ('beginn ','begins '),('spilt','spilled'),('deni ','denies '),('consignation ','consignment '),('trampl ','trample '),('unstop ','unseal '),('suppos ','supposes '),('dispis ','despise '),
           ('stirr ','stirs '),('harv ','harvest '),('satisfi ','satisfies '),('advocateth ','advocates '),('manif ','manifest '),('hindereth ','hinder '),('forgett ','forget '),('reigneth ','reigns '),
           ('fainteth','faints '),('defence ','defense '),('hubl ','hubles '),('shouldst ','should '),('acknowledg ','acknowledges '),('meaneth ','means '),('comfortedst ','comforted '),('griev ','grieves '),
           ('methought ','I thought '),('invit ','invites '),('vainness ','vanity '),('shedd ','sheds '),('cleav ','cleaves '),('fulfil ','fulfill '),('choic ','well chosen '),('endur ','endures '),('smit ','smites '),
           ('allott ','allots '),('declar ','declares '),('sepulchre ','tomb '),('saidst ','said '),('Believ ','believe '),('Canst ','cannot ')]
#put everything to lowercase?

def clean_text(text): #removes punctuation, html tags, and pronouns that are not listed above
#     clean = TAG_REMOVE.sub('', text)
    clean = text.replace('<i>', '')
    clean = clean.replace('</i>','')
    clean = " ".join(re.compile('\w+').findall(clean))
    if 'LORD' in clean:
        clean = clean.replace('LORD','lord')
    clean = " ".join(clean.split('eth '))
    clean = " ".join(clean.split('est '))
    for word in stopwords.words('english'):
        if word not in model:
            clean = clean.replace(' '+ word +' ', ' ')
    for k, v in mapping: #replace words that shouldn't have 'eth' or 'est' taken off
        clean = clean.replace(k, v)
    return clean

# bibles = pd.read_csv( 'bibles.csv' )
# bible_file = bibles[["Verse", "King James Bible"]]

# bible = {}
# #Example: bible['Matthew'][1][1] is Matthew 1:1
# for i in range(len(bible_file)):
#     book_chp_vrs = bible_file["Verse"][i].split()
#     book = " ".join(book_chp_vrs[:-1])
#     chap, verse = book_chp_vrs[-1].split(":")
#     chap = int(chap)
#     verse = int(verse)
#     text = bible_file["King James Bible"][i]
#     if book not in bible.keys():
#         if chap == 1 and verse == 1:
#             bible[book] = [[],['',clean_text(text)]] #insert clean text
#         else:
#             print("Error, not chapter 1")
#     else:
#         if len(bible[book]) == chap:
#             bible[book].append([clean_text(text)])
#         else:
            
#             bible[book][chap].append(clean_text(text)) #insert clean text 
            
# # for key in bible.keys():
# #     for chapter in bible[key]:
# #         for verse in chapter:
# #             print(verse)

# print("bible done")

## Get BoM into dictionary

In [3]:
import re


def read_chapter(file, line, BoM):
    blank = next(file)
    line = next(file)
    while True: #Read each verse
        if line == 'THE END':
            break
        
        if chap_header_pattern.match(line):
            while not book_chp_vrs_pattern.match(line):
                line = next(file)
        book_chp_vrs = line.split()
        book = " ".join(book_chp_vrs[:-1])
        chap, verse = book_chp_vrs[-1].split(":")
        chap = int(chap)
        verse = int(verse)
        text = next(file).split()
        text = " ".join(text[1:])
        text = text.replace('\n','') + ' '
        line = text
        count = 0
        while True: #Read full text of verse
            count += 1
            line = next(file)
            if line == '\n':
                while not book_chp_vrs_pattern.match(line) and line != 'THE END':
                    line = next(file)
                break
            text += line.replace('\n', ' ')
            if count > 100:
                print("100 error")
                break
        #Put verse in dicionary
        if book not in BoM.keys():
            if chap == 1 and verse == 1:
                BoM[book] = [[],['',clean_text(text).split()]] #insert clean text
            else:
                print("Error, not chapter 1")
        else:
            if len(BoM[book]) == chap:
                BoM[book].append([clean_text(text).split()])
            else:
                BoM[book][chap].append(clean_text(text).split()) #insert clean text
            

pattern = re.compile("(Chapter \d+)")
book_chp_vrs_pattern = re.compile("(\d*( )*([A-Za-z]+ )*\d+:\d+)")
chap_header_pattern = re.compile("(\d*( )*([A-Za-z]+ )*\d+$)")

bom_file = open('book_of_mormon.txt')
BoM = {}
for line in bom_file:
    if pattern.match(line):
#         line, bom_file = read_chapter(bom_file, line)
        read_chapter(bom_file, line, BoM)
        
print("BoM done")

BoM done


## Convert BoM to vectors  

In [24]:
import copy

unknown_words = {''}
for key in BoM.keys():
    for chapter in range(len(BoM[key])):
        for verse in BoM[key][chapter]:
            for word in verse:
                if word not in model:
                    unknown_words.add(word)       
unknown_words.remove('')

# print(unknown_words)
# print(len(unknown_words))


def average_unknown_word(word_ind, verse):
    if word_ind - 2 < 0:
        begin = 0
    else:
        begin = word_ind - 2
    if word_ind + 2 > len(verse):
        end = len(verse)
    else:
        end = word_ind + 2
    numVerses = 0
    vec = np.zeros((300,))
    for word in range(begin,end):
        if type(verse[word]) != type('str'):
            vec += verse[word]
            numVerses += 1
    if numVerses != 0:
        vec /= numVerses
    else:
        print("zero error")
    return vec

vec_BoM = copy.deepcopy(BoM)

for key in BoM.keys():
    for chapter in range(1,len(BoM[key])):
        for verse in range(1,len(BoM[key][chapter])):
            for word in range(1,len(BoM[key][chapter][verse])):
                cur_word = BoM[key][chapter][verse][word]
                if cur_word not in unknown_words:
                    vec_BoM[key][chapter][verse][word] = model[cur_word]

print('known words done')

for key in BoM.keys():
    for chapter in range(1,len(BoM[key])):
        for verse in range(1,len(BoM[key][chapter])):
            for word in range(1,len(BoM[key][chapter][verse])):
                cur_word = BoM[key][chapter][verse][word]
                if cur_word in unknown_words:
                    vec_BoM[key][chapter][verse][word] = average_unknown_word(word, vec_BoM[key][chapter][verse])
print("done")

known words done
done


In [25]:
for key in BoM.keys():
    for chapter in range(1,len(BoM[key])):
        for verse in range(1,len(BoM[key][chapter])):
            for word in range(1,len(BoM[key][chapter][verse])):
                cur_word = BoM[key][chapter][verse][word]
                if type(vec_BoM[key][chapter][verse][word]) == type('str'):
                    print(vec_BoM[key][chapter][verse][word])
print('done')

done


In [1]:
import multiprocessing
from gensim.corpora.wikicorpus import WikiCorpus
from gensim.models.word2vec import Word2Vec
from gensim.models import word2vec
from nltk.corpus import gutenberg
from string import punctuation


sents = gutenberg.sents('bible-kjv.txt')
for file in gutenberg.fileids():
    if file != 'bible-kjv.txt':
        sents += gutenberg.sents(file)

# for sent in sents:
#     print(sent)

# print(len(bible_kjv_sents))
# print(len(sents))

discard_punctuation_and_lowercased_sents = [[word.lower() for word in sent if word not in punctuation] for sent in sents]
# # bible_model = word2vec.Word2Vec(discard_punctuation_and_lowercased_sents, min_count=5, size=300)

print('bible done')
wiki = WikiCorpus('data/sw/swwiki-latest-pages-articles.xml.bz2', 
                  lemmatize=False, dictionary={})
sentences = list(wiki.get_texts())
sents = sentences + discard_punctuation_and_lowercased_sents
print('len')
print(len(sents))
params = {'size': 300, 'window': 10, 'min_count': 10, 
          'workers': max(1, multiprocessing.cpu_count() - 1), 'sample': 1E-3,}
wiki_model = Word2Vec(sents, **params).wv
print('done')

bible done
len
120697
done
